In [ ]:
# GPU Acceleration Example
import cupy as cp  # For GPU acceleration
import numpy as np
import time

def gpu_accelerated_preprocessing(system_matrices, num_iterations):
    """
    Example of GPU-accelerated matrix operations for system analysis
    """
    # Move data to GPU
    A_gpu = cp.array(system_matrices['A'])
    B_gpu = cp.array(system_matrices['B'])
    
    # Perform operations on GPU
    start_time = time.time()
    for _ in range(num_iterations):
        # Example: Computing reachable states
        result = cp.matmul(A_gpu, B_gpu)
        
    # Move result back to CPU
    result_cpu = cp.asnumpy(result)
    end_time = time.time()
    
    print(f"GPU computation time: {end_time - start_time:.4f} seconds")
    return result_cpu

# Example usage
if __name__ == "__main__":
    # Sample system matrices
    A = np.random.rand(1000, 1000)
    B = np.random.rand(1000, 100)
    
    system_matrices = {'A': A, 'B': B}
    result = gpu_accelerated_preprocessing(system_matrices, 100)

# MATLAB to FlowStar Model Generator

This notebook provides utilities to convert MATLAB system models into FlowStar model files. It handles:
- System dynamics from MATLAB state-space models
- Safety specifications
- Initial conditions
- Guard conditions
- Reset functions

## Usage
1. Prepare your MATLAB model file (.m or .mat)
2. Define safety properties and initial conditions
3. Run the conversion process
4. Generate FlowStar model file

In [ ]:
import numpy as np
from typing import List, Tuple
import os

def generate_flowstar_model(
    Ac: np.ndarray,  # Continuous-time A matrix
    Bc: np.ndarray,  # Continuous-time B matrix
    Cc: np.ndarray,  # Continuous-time C matrix
    Dc: np.ndarray,  # Continuous-time D matrix
    K: np.ndarray,   # Discrete-time controller gain
    L: np.ndarray,   # Discrete-time observer gain
    num_locations: int,  # Number of locations in the hybrid automaton
    length: int,     # Length of the (m,k)-weakly hard pattern
    miss_count: int, # Number of allowed misses in the pattern
    output_file: str, # Output .model file path
    initial_states: List[Tuple[float, float]] = None,  # List of (min, max) for each state
    input_bounds: List[Tuple[float, float]] = None,    # List of (min, max) for each input
    sampling_time: float = 0.01,  # Sampling time for the discrete controller
    time_horizon: float = 10.0    # Total simulation time
) -> None:
    """
    Generate a Flowstar model file for a hybrid system with weakly-hard constraints.
    
    Parameters:
    -----------
    Ac, Bc, Cc, Dc : numpy.ndarray
        System matrices in continuous time
    K : numpy.ndarray
        Discrete-time controller gain
    L : numpy.ndarray
        Discrete-time observer gain
    num_locations : int
        Number of locations in the hybrid automaton
    length : int
        Length of the (m,k)-weakly hard pattern
    miss_count : int
        Number of allowed misses in the pattern
    output_file : str
        Path to save the generated .model file
    initial_states : List[Tuple[float, float]], optional
        Initial state bounds for each state variable
    input_bounds : List[Tuple[float, float]], optional
        Input bounds for each input
    sampling_time : float, optional
        Sampling time of the discrete controller
    time_horizon : float, optional
        Total simulation time
    """

In [ ]:
def generate_state_vars(n_states: int) -> str:
    """Generate state variable declarations"""
    return '\n'.join([f'state var x{i}' for i in range(n_states)])

def generate_mode_vars(n_modes: int) -> str:
    """Generate mode variable declarations"""
    return 'discrete var mode, pattern_pos\n'

def matrix_to_flowstar(matrix: np.ndarray, prefix: str = '') -> str:
    """Convert a numpy matrix to Flowstar format"""
    rows, cols = matrix.shape
    lines = []
    for i in range(rows):
        terms = []
        for j in range(cols):
            val = matrix[i, j]
            if abs(val) > 1e-10:  # Ignore very small values
                if j == 0:
                    terms.append(f'{val}')
                else:
                    terms.append(f'{val}*{prefix}{j}')
        lines.append(' + '.join(terms) if terms else '0')
    return '\n'.join(lines)

def generate_init_section(initial_states: List[Tuple[float, float]]) -> str:
    """Generate the init section"""
    conditions = []
    for i, (min_val, max_val) in enumerate(initial_states):
        conditions.append(f'x{i} in [{min_val}, {max_val}]')
    return 'init\n{\n\tmode == 0\n\t' + '\n\t'.join(conditions) + '\n}'

def generate_continuous_dynamics(Ac: np.ndarray, Bc: np.ndarray) -> str:
    """Generate continuous dynamics equations"""
    n_states = Ac.shape[0]
    dynamics = []
    
    # Convert continuous-time state equations
    state_eqs = Ac @ np.array([f'x{i}' for i in range(n_states)]).reshape(-1, 1)
    input_terms = Bc @ np.array([f'u{i}' for i in range(Bc.shape[1])]).reshape(-1, 1)
    
    for i in range(n_states):
        dynamics.append(f"x{i}' = {state_eqs[i, 0]}")
        if Bc.shape[1] > 0:
            dynamics.append(f" + {input_terms[i, 0]}")
            
    return '\n'.join(dynamics)

In [ ]:
def generate_flowstar_model(
    Ac: np.ndarray,
    Bc: np.ndarray,
    Cc: np.ndarray,
    Dc: np.ndarray,
    K: np.ndarray,
    L: np.ndarray,
    num_locations: int,
    length: int,
    miss_count: int,
    output_file: str,
    initial_states: List[Tuple[float, float]] = None,
    input_bounds: List[Tuple[float, float]] = None,
    sampling_time: float = 0.01,
    time_horizon: float = 10.0
) -> None:
    n_states = Ac.shape[0]
    n_inputs = Bc.shape[1]
    
    if initial_states is None:
        initial_states = [(0.0, 0.0)] * n_states
    
    if input_bounds is None:
        input_bounds = [(-1.0, 1.0)] * n_inputs
        
    # Generate the model file content
    model_content = []
    
    # Variable declarations
    model_content.append("# State variables")
    model_content.append(generate_state_vars(n_states))
    model_content.append("\n# Mode variables")
    model_content.append(generate_mode_vars(num_locations))
    
    # Settings
    model_content.append("\nsetting")
    model_content.append("{")
    model_content.append(f"\tfixed steps {sampling_time}")
    model_content.append(f"\ttime {time_horizon}")
    model_content.append("\tremainder estimation 1e-4")
    model_content.append("\tidentity precondition")
    model_content.append("\tgnuplot octagon x0,x1")
    model_content.append("\toutput out")
    model_content.append("}")
    
    # Initial conditions
    model_content.append("\n" + generate_init_section(initial_states))
    
    # Generate locations (modes)
    for loc in range(num_locations):
        model_content.append(f"\n# Location {loc}")
        model_content.append(f"loc l{loc}")
        model_content.append("{")
        
        # Continuous dynamics
        model_content.append("\tdynamic")
        model_content.append("\t{")
        dynamics = generate_continuous_dynamics(Ac, Bc)
        for line in dynamics.split('\n'):
            model_content.append(f"\t\t{line}")
        model_content.append("\t}")
        
        # Invariant conditions
        model_content.append("\tinv")
        model_content.append("\t{")
        model_content.append(f"\t\tmode == {loc}")
        model_content.append("\t}")
        
        model_content.append("}")
        
    # Generate transitions
    for loc in range(num_locations):
        next_loc = (loc + 1) % num_locations
        model_content.append(f"\n# Transition from l{loc} to l{next_loc}")
        model_content.append(f"trans")
        model_content.append("{")
        model_content.append(f"\tfrom: l{loc}")
        model_content.append(f"\tto: l{next_loc}")
        model_content.append("\tguard { pattern_pos == 0 }")
        model_content.append("\treset {")
        model_content.append(f"\t\tmode' := {next_loc}")
        model_content.append("\t\tpattern_pos' := pattern_pos + 1")
        model_content.append("\t}")
        model_content.append("}")
    
    # Write the model file
    with open(output_file, 'w') as f:
        f.write('\n'.join(model_content))

In [ ]:
# Example usage
if __name__ == "__main__":
    # Example system matrices (2-state system)
    Ac = np.array([
        [0, 1],
        [-2, -3]
    ])
    
    Bc = np.array([
        [0],
        [1]
    ])
    
    Cc = np.array([
        [1, 0]
    ])
    
    Dc = np.array([[0]])
    
    # Example controller and observer gains
    K = np.array([[1, 2]])
    L = np.array([[0.5], [1.0]])
    
    # Weakly-hard parameters
    num_locations = 4  # Number of locations in automaton
    length = 5        # Length of pattern
    miss_count = 2    # Number of allowed misses
    
    # Initial state bounds
    initial_states = [(-1, 1), (-1, 1)]  # For x0 and x1
    
    # Generate the model file
    generate_flowstar_model(
        Ac=Ac,
        Bc=Bc,
        Cc=Cc,
        Dc=Dc,
        K=K,
        L=L,
        num_locations=num_locations,
        length=length,
        miss_count=miss_count,
        output_file="example_model.model",
        initial_states=initial_states,
        sampling_time=0.01,
        time_horizon=10.0
    )
    
print("Model file has been generated!")

UFuncTypeError: ufunc 'matmul' did not contain a loop with signature matching types (dtype('int64'), dtype('<U2')) -> None

In [ ]:
# Required Libraries
import numpy as np
import scipy.io as sio
from pathlib import Path
import re
import sympy as sp
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple, Union
import matlab.engine  # For direct MATLAB integration

In [ ]:
# Data Classes for System Components
@dataclass
class StateVariable:
    name: str
    initial_range: Tuple[float, float]
    unsafe_range: Optional[Tuple[float, float]] = None
    
@dataclass
class SystemParameters:
    name: str
    A: np.ndarray  # System matrix
    B: np.ndarray  # Input matrix
    C: np.ndarray  # Output matrix
    D: np.ndarray  # Feedthrough matrix
    state_vars: List[StateVariable]
    sampling_time: float
    max_time: float
    
@dataclass
class Guard:
    condition: str
    from_mode: str
    to_mode: str
    reset: Dict[str, str]
    
@dataclass
class Mode:
    name: str
    dynamics: Dict[str, str]
    invariants: List[str]

In [ ]:
def parse_matlab_model(file_path: str) -> SystemParameters:
    """
    Parse MATLAB model file and extract system parameters
    
    Args:
        file_path: Path to .m or .mat file
        
    Returns:
        SystemParameters object containing model information
    """
    # Initialize MATLAB engine
    eng = matlab.engine.start_matlab()
    
    try:
        if file_path.endswith('.mat'):
            # Load .mat file
            mat_contents = sio.loadmat(file_path)
            A = mat_contents.get('A', None)
            B = mat_contents.get('B', None)
            C = mat_contents.get('C', None)
            D = mat_contents.get('D', None)
        else:
            # Execute .m file
            eng.addpath(str(Path(file_path).parent))
            eng.run(str(Path(file_path).stem), nargout=0)
            
            # Get variables from MATLAB workspace
            A = np.array(eng.workspace['A'])
            B = np.array(eng.workspace['B'])
            C = np.array(eng.workspace['C'])
            D = np.array(eng.workspace['D'])
            
        # Extract state variable names if available
        state_vars = []
        n_states = A.shape[0]
        for i in range(n_states):
            state_vars.append(StateVariable(
                name=f'x{i+1}',
                initial_range=(-1.0, 1.0)  # Default range
            ))
            
        return SystemParameters(
            name=Path(file_path).stem,
            A=A, B=B, C=C, D=D,
            state_vars=state_vars,
            sampling_time=0.01,  # Default sampling time
            max_time=1.0  # Default max time
        )
        
    finally:
        eng.quit()

In [ ]:
def generate_dynamics_equations(system: SystemParameters) -> Dict[str, str]:
    """
    Generate symbolic equations for system dynamics
    """
    dynamics = {}
    x = sp.Matrix(sp.symbols(f'x1:{len(system.state_vars)+1}'))
    
    # Convert numpy arrays to symbolic matrices
    A = sp.Matrix(system.A)
    
    # Generate dynamics equations
    dx = A * x
    
    for i, state_var in enumerate(system.state_vars):
        dynamics[f"{state_var.name}'"] = str(dx[i])
        
    return dynamics

def generate_mode(name: str, dynamics: Dict[str, str], invariants: List[str]) -> str:
    """
    Generate FlowStar mode description
    """
    mode_str = f"  {name}\n  {{\n   poly ode 1\n   {{\n"
    
    # Add dynamics equations
    for var, eq in dynamics.items():
        mode_str += f"    {var} = {eq}\n"
    
    mode_str += "   }\n   inv\n   {\n"
    
    # Add invariants
    for inv in invariants:
        mode_str += f"    {inv}\n"
    
    mode_str += "   }\n  }\n"
    
    return mode_str

In [ ]:
def generate_flowstar_model(system: SystemParameters, 
                          modes: List[Mode],
                          guards: List[Guard]) -> str:
    """
    Generate complete FlowStar model file content
    """
    model = f"hybrid reachability\n{{\n"
    
    # State variables declaration
    state_vars = [var.name for var in system.state_vars]
    model += f" state var {', '.join(state_vars)}\n\n"
    
    # Parameters section
    model += " setting\n {\n"
    model += f"  fixed steps {system.sampling_time}\n"
    model += f"  time {system.max_time}\n"
    model += "  remainder estimation 1e-3\n"
    model += "  identity precondition\n"
    model += f"  output {system.name}\n"
    model += "  max jumps 100\n"
    model += "  print on\n }\n\n"
    
    # Modes section
    model += " modes\n {\n"
    for mode in modes:
        model += generate_mode(mode.name, mode.dynamics, mode.invariants)
    model += " }\n\n"
    
    # Jumps section
    model += " jumps\n {\n"
    for guard in guards:
        model += generate_guard(guard)
    model += " }\n\n"
    
    # Initial conditions
    model += " init\n {\n"
    model += "  l1\n  {\n"
    for var in system.state_vars:
        model += f"   {var.name} in [{var.initial_range[0]},{var.initial_range[1]}]\n"
    model += "  }\n }\n"
    
    # Unsafe conditions
    model += "}\n\nunsafe\n{\n"
    for mode in modes:
        model += f"  {mode.name}\n  {{\n"
        for var in system.state_vars:
            if var.unsafe_range:
                model += f"   {var.name} >= {var.unsafe_range[1]}  {var.name} <= {var.unsafe_range[0]}\n"
        model += "  }\n"
    model += "}"
    
    return model

def generate_guard(guard: Guard) -> str:
    """
    Generate FlowStar guard description
    """
    guard_str = f"  {guard.from_mode} -> {guard.to_mode}\n"
    guard_str += f"  guard {{ {guard.condition} }}\n"
    guard_str += "  reset {\n"
    for var, reset in guard.reset.items():
        guard_str += f"    {var}' := {reset}\n"
    guard_str += "  }\n"
    return guard_str

# Example Usage

Let's demonstrate how to use this converter with a sample MATLAB system model.

In [ ]:
# Example usage with a sample system
def example_conversion():
    # Sample system parameters (normally these would come from MATLAB file)
    A = np.array([
        [0.995860265603438, 0.0378696105301410],
        [-0.174562623798343, 0.908578953704267]
    ])
    B = np.array([[1.0], [0.5]])
    C = np.eye(2)
    D = np.zeros((2, 1))
    
    # Create state variables
    state_vars = [
        StateVariable(
            name='x1',
            initial_range=(-1, 1),
            unsafe_range=(-2, 2)
        ),
        StateVariable(
            name='x2',
            initial_range=(-100, 100),
            unsafe_range=(-200, 200)
        )
    ]
    
    # Create system parameters
    system = SystemParameters(
        name="sample_system",
        A=A, B=B, C=C, D=D,
        state_vars=state_vars,
        sampling_time=0.01,
        max_time=1.0
    )
    
    # Generate dynamics
    dynamics = generate_dynamics_equations(system)
    
    # Create modes
    modes = [
        Mode(
            name="l1",
            dynamics=dynamics,
            invariants=["length <= 20", "missct <= 10"]
        )
    ]
    
    # Create guards
    guards = [
        Guard(
            from_mode="l1",
            to_mode="l1",
            condition="length + 1 <= 20 and missct <= 10 and dt = 0.04",
            reset={
                "x1": "0.242485063948108*x1 + 0.019880697758022*x2",
                "x2": "-4.966153983486504*x1 + 0.236421716395938*x2"
            }
        )
    ]
    
    # Generate FlowStar model
    model_content = generate_flowstar_model(system, modes, guards)
    
    # Save to file
    output_path = "models/sample_system.model"
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, "w") as f:
        f.write(model_content)
    
    return model_content

# Run the example
model_content = example_conversion()
print("Generated model content:")
print(model_content)

In [ ]:
def generate_specialized_model(
    mat_file: str,
    output_prefix: str,
    h: float,  # sampling time
    mdadt1: int,  # MDADT parameter for location 1
    Kmax_values: List[int],  # List of maximum length parameters to try
    mmax_values: List[int],  # List of maximum miss counts to try
    init_ranges: List[List[Tuple[float, float]]]  # List of initial condition ranges for state variables
) -> None:
    """
    Generate specialized FlowStar model files with different parameters and initial conditions.
    
    Args:
        mat_file: Path to .mat file containing system matrices
        output_prefix: Prefix for output model files
        h: Sampling time
        mdadt1: MDADT parameter for location 1
        Kmax_values: List of K_max values to try
        mmax_values: List of m_max values to try
        init_ranges: List of initial condition ranges for different configurations
    """
    # Load system matrices from .mat file
    mat_data = sio.loadmat(mat_file)
    A = mat_data['A']
    B = mat_data['B']
    C = mat_data['C']
    D = mat_data.get('D', np.zeros((C.shape[0], B.shape[1])))
    
    n_states = A.shape[0]  # Number of physical states
    n_total = 2 * n_states  # Including estimated states
    
    # Generate models for each combination
    for Kmax in Kmax_values:
        for mmax in mmax_values:
            for init_idx, init_range in enumerate(init_ranges):
                model_content = []
                
                # File name encoding parameters
                file_name = f"{output_prefix}_K{Kmax}_M{mmax}_I{init_idx}.model"
                
                # Header
                model_content.append("hybrid reachability\n{")
                
                # State variables
                states = [f"x{i+1}" for i in range(n_states)]  # Physical states
                est_states = [f"x{i+1}_hat" for i in range(n_states)]  # Estimated states
                model_content.append(f" state var {', '.join(states + est_states)}, dt, gt, length, missct")
                
                # Parameters section
                model_content.append(f"\n par\n {{")
                model_content.append(f"  Kmax = {Kmax}  mmax = {mmax}  h = {h}  mdadt1 = {mdadt1}")
                
                # Add system matrices as parameters
                for i in range(n_states):
                    for j in range(n_states):
                        model_content.append(f"  a{i+1}{j+1} = {A[i,j]}")
                # Add B, C matrices similarly
                model_content.append(" }")

                # Settings
                model_content.extend([
                    "\n setting",
                    " {",
                    f"  fixed steps {h/10}",  # Smaller steps for accurate simulation
                    f"  time {Kmax * h}",
                    "  remainder estimation 1e-3",
                    "  identity precondition",
                    f"  gnuplot octagon gt, x1",
                    "  fixed orders 12",
                    "  cutoff 1e-8",
                    "  precision 53",
                    f"  output {output_prefix}_K{Kmax}_M{mmax}_I{init_idx}",
                    f"  max jumps {Kmax * 2}",
                    "  print on",
                    " }\n"
                ])

In [ ]:
                # Generate modes
                model_content.append(" modes\n {")
                
                # Initial mode l0 with zero dynamics
                model_content.extend([
                    "  l0",
                    "  {",
                    "   poly ode 1",
                    "   {",
                    *[f"    {var}' = 0" for var in states + est_states],
                    "    dt' = 0",
                    "    gt' = 1",
                    "    length' = 0",
                    "    missct' = 0",
                    "   }",
                    "   inv",
                    "   {",
                    "    gt = 0",
                    "   }",
                    "  }"
                ])
                
                # Mode l1 (successful update)
                model_content.extend([
                    "\n  l1",
                    "  {",
                    "   poly ode 1",
                    "   {",
                    *generate_continuous_dynamics(A, B, K, states, est_states),
                    "    dt' = h",
                    "    gt' = 1",
                    "    length' = 0",
                    "    missct' = 0",
                    "   }",
                    "   inv",
                    "   {",
                    "    length <= Kmax",
                    "    missct <= mmax",
                    "   }",
                    "  }"
                ])
                
                # Mode l1dt (MDADT handling)
                model_content.extend([
                    "\n  l1dt",
                    "  {",
                    "   poly ode 1",
                    "   {",
                    *generate_continuous_dynamics(A, B, K, states, est_states),
                    "    dt' = h",
                    "    gt' = 1",
                    "    length' = 0",
                    "    missct' = 0",
                    "   }",
                    "   inv",
                    "   {",
                    "    length <= Kmax",
                    "    missct <= mmax",
                    "   }",
                    "  }"
                ])
                
                # Generate l10, l100, etc. modes for missed updates
                for i in range(1, min(mmax + 1, Kmax)):
                    mode_name = f"l{'1' + '0' * i}"
                    model_content.extend([
                        f"\n  {mode_name}",
                        "  {",
                        "   poly ode 1",
                        "   {",
                        *generate_continuous_dynamics(A, B, K, states, est_states),
                        "    dt' = h",
                        "    gt' = 1",
                        "    length' = 0",
                        "    missct' = 0",
                        "   }",
                        "   inv",
                        "   {",
                        "    length <= Kmax",
                        "    missct <= mmax",
                        "   }",
                        "  }"
                    ])

In [ ]:
                # Generate jumps
                model_content.append("\n jumps\n {")
                
                # Jumps from l0 to other modes
                for i in range(min(mmax + 1, Kmax)):
                    mode_name = f"l{'1' + '0' * i}"
                    model_content.extend([
                        f"  l0 -> {mode_name}",
                        "  guard { gt = 0 }",
                        "  reset {",
                        *generate_state_resets(A, B, K, states, est_states),
                        f"    length' := {i + 1}",
                        f"    missct' := {i}",
                        "    dt' := 0",
                        "  }",
                        "  interval aggregation\n"
                    ])
                
                # Jumps from l1 (successful update)
                for i in range(min(mmax + 1, Kmax)):
                    mode_name = f"l{'1' + '0' * i}"
                    model_content.extend([
                        f"  l1 -> {mode_name}",
                        f"  guard {{ length + {i + 1} <= Kmax  missct + {i} <= mmax  dt = h }}",
                        "  reset {",
                        *generate_state_resets(A, B, K, states, est_states),
                        f"    length' := length + {i + 1}",
                        f"    missct' := missct + {i}",
                        "    dt' := 0",
                        "  }",
                        "  interval aggregation\n"
                    ])
                
                # Jumps for l1dt (MDADT handling)
                model_content.extend([
                    "  l1dt -> l1dt",
                    f"  guard {{ dt = h  gt <= {mdadt1 * h} }}",
                    "  reset {",
                    *generate_state_resets(A, B, K, states, est_states),
                    "    length' := length",
                    "    missct' := missct",
                    "    dt' := 0",
                    "  }",
                    "  interval aggregation\n"
                ])
                
                # Generate init section
                model_content.append("\n init\n {")
                model_content.append("  l0\n  {")
                for i, (min_val, max_val) in enumerate(init_range):
                    model_content.append(f"   x{i+1} in [{min_val},{max_val}]")
                    model_content.append(f"   x{i+1}_hat in [{min_val},{max_val}]")
                model_content.extend([
                    "   dt in [0,0]",
                    "   gt in [0,0]",
                    "   length in [0,0]",
                    "   missct in [0,0]",
                    "  }",
                    " }"
                ])
                
                # Generate unsafe section
                model_content.append("\nunsafe\n{")
                for mode in ["l1", "l1dt"] + [f"l{'1' + '0' * i}" for i in range(1, min(mmax + 1, Kmax))]:
                    model_content.append(f"  {mode}")
                    model_content.append("  {")
                    for i in range(n_states):
                        unsafe_range = [-2, 2]  # Example unsafe ranges, adjust as needed
                        model_content.append(f"    x{i+1} >= {unsafe_range[1]}  x{i+1} <= {unsafe_range[0]}")
                    model_content.append("  }")
                model_content.append("}")
                
                # Write to file
                with open(file_name, 'w') as f:
                    f.write('\n'.join(model_content))
                
def generate_continuous_dynamics(A, B, K, states, est_states):
    """Generate continuous dynamics equations"""
    n_states = A.shape[0]
    dynamics = []
    
    # Physical states
    for i in range(n_states):
        eq = [f"{A[i,j]}*{states[j]}" for j in range(n_states)]
        est_terms = [f"{-B[i,0]*K[0,j]}*{est_states[j]}" for j in range(n_states)]
        dynamics.append(f"    {states[i]}' = {' + '.join(eq + est_terms)}")
    
    # Estimated states
    for i in range(n_states):
        dynamics.append(f"    {est_states[i]}' = 0")
        
    return dynamics

def generate_state_resets(A, B, K, states, est_states):
    """Generate reset equations for state updates"""
    n_states = A.shape[0]
    resets = []
    
    # Reset estimated states based on current physical states
    for i in range(n_states):
        eq = [f"{A[i,j]}*{est_states[j]}" for j in range(n_states)]
        state_terms = [f"{B[i,0]}*{states[0]}"]  # Assuming single input
        resets.append(f"    {est_states[i]}' := {' + '.join(eq + state_terms)}")
        
    return resets

In [ ]:
# Example usage
if __name__ == "__main__":
    # Example parameters
    mat_file = "models/system_with_noise.mat"
    output_prefix = "models/sus_ctrl/SUS"
    h = 0.04  # sampling time
    mdadt1 = 4  # MDADT parameter
    
    # Parameters to try
    Kmax_values = [10, 15, 20]
    mmax_values = [5, 8, 10]
    
    # Different initial conditions to try
    init_ranges = [
        [(-1, 1), (-100, 100), (-50, 50), (-300, 300)],  # First configuration
        [(-0.5, 0.5), (-50, 50), (-25, 25), (-150, 150)],  # Second configuration
        [(-2, 2), (-200, 200), (-100, 100), (-600, 600)]   # Third configuration
    ]
    
    # Generate all model files
    generate_specialized_model(
        mat_file=mat_file,
        output_prefix=output_prefix,
        h=h,
        mdadt1=mdadt1,
        Kmax_values=Kmax_values,
        mmax_values=mmax_values,
        init_ranges=init_ranges
    )
    
    print("Model files generated successfully!")